In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys


In [ ]:

# ARGV
MACHINE = "GPUs"#str(sys.argv[1])
BATCHING = 2000#int(sys.argv[2])
SYSTEM_SIZE = 4#int(sys.argv[3])
COUPLING = 0.5000#float(sys.argv[4])
SHIFT = 6.28#float(sys.argv[])
FEET = 1000#int(sys.argv[])
DT = 0.005#float(sys.argv[])
LENGTH_IN = 200#int(sys.argv[5])
DIM = int(2 * SYSTEM_SIZE)
LENGTH_TEST = 5000#int(0.025 * LENGTH_MAX)#int(sys.argv[])
LABEL = 1#int(sys.argv[6])
MAX_EPOCH = 2 #int(sys.argv[7])
TRIAL = 1 #int(sys.argv[8])





In [ ]:

dir_root = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/'
dir_save = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/'

if not os.path.exists(dir_root):
    os.mkdir(dir_root)
if not os.path.exists(dir_save.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN)):
    os.mkdir(dir_save.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN))


In [ ]:

def ann(label):
    input = tf.keras.Input(shape = (DIM * LENGTH_IN,), dtype = tf.float32)

    if label == 1:
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(input)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
    elif label == 2:
        x = tf.keras.layers.Reshape((LENGTH_IN, DIM,), input_shape=(DIM * LENGTH_IN,))(input)
        x = tf.keras.layers.Conv1D(2 * DIM, 2, activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Conv1D(2 * DIM, 2, activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
    elif label == 3:
        x = tf.keras.layers.Reshape((LENGTH_IN, SYSTEM_SIZE, 2,), input_shape=(DIM * LENGTH_IN,))(input)
        x = tf.keras.layers.Conv2D(2 * DIM, (3, 2), activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Conv2D(2 * DIM, (3, 2), activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
        x = tf.keras.layers.Dense(2 * DIM, activation = tf.keras.activations.tanh)(x)
    elif label == 4:
        x = tf.keras.layers.Reshape((LENGTH_IN, DIM,), input_shape=(DIM * LENGTH_IN,))(input)
        x = tf.keras.layers.LSTM(2 * DIM)(x)
    elif label == 5:
        x = tf.keras.layers.Reshape((LENGTH_IN, DIM,), input_shape=(DIM * LENGTH_IN,))(input)
        x = tf.keras.layers.LSTM(2 * DIM, return_sequences = True)(x)
        x = tf.keras.layers.LSTM(2 * DIM)(x)

        
    output = tf.keras.layers.Dense(DIM, activation=tf.keras.activations.linear)(x)

    model = tf.keras.Model(input, output)
    model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4), loss = tf.keras.losses.mean_squared_error)

    return model


In [ ]:
#parameters depends on machine
if MACHINE == 'GPUs':
    os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
    strategy = tf.distribute.MirroredStrategy()
    # strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    # strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice())
    BATCH_SIZE_REP = BATCHING
    BATCH_SIZE = BATCH_SIZE_REP * strategy.num_replicas_in_sync
    SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 4
    def make_model(label):
        with strategy.scope():
            return ann(label)
elif MACHINE == 'CPUs':
    BATCH_SIZE = BATCHING
    SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 4
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    def make_model(label):
        return ann(label)
elif MACHINE == 'GPU':
    BATCH_SIZE = BATCHING
    SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 4
    def make_model(label):
        return ann(label)
else:
    print("Select MACHINE : CPUs/GPUs/GPU")
    raise SystemExit


In [ ]:
file = '/pds/pds121/KMML/train_prediction/Phaset_w{w:0.2f}_N{N:d}_K{K:0.4f}_ft{ft:d}_dt{dt:0.3f}_E{e:d}.dat'


In [ ]:

def make_batch(low, high, batch_size):
    data = np.loadtxt(file.format(N = SYSTEM_SIZE, K = COUPLING, w = SHIFT, ft = FEET, dt = DT, e = 1))    
    images = np.zeros((batch_size, DIM * LENGTH_IN), dtype=np.float32)
    labels = np.zeros((batch_size, DIM), dtype=np.float32)
    shuffle = np.random.randint(low, high, batch_size)

    for i in np.arange(batch_size):
        images[i] = np.reshape(data[shuffle[i]:shuffle[i] + LENGTH_IN], (DIM * LENGTH_IN))
        labels[i] = data[shuffle[i] + LENGTH_IN]

    dataset_images = tf.data.Dataset.from_tensor_slices(images)
    dataset_labels = tf.data.Dataset.from_tensor_slices(labels)

    dataset =  tf.data.Dataset.zip((dataset_images, dataset_labels))
    dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
  
    return dataset


In [ ]:
struct = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_struct.png'


In [ ]:
model = make_model(LABEL)

In [ ]:
tf.keras.utils.plot_model(model, to_file = struct.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL), show_shapes = True, show_layer_names = False, rankdir = 'LR')


In [ ]:
VALID_INI = 180001
VALID_END = 190000
VALID_SIZE = VALID_END - VALID_INI + 1
TRAIN_INI = 1
TRAIN_END = 180000
TRAIN_SIZE = TRAIN_END - TRAIN_INI + 1

In [ ]:
history_pdf = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_history_{trial:d}.pdf'
history_dat = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_history_{trial:d}.dat'
save = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_save_{trial:d}'
if not os.path.exists(save.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, trial = TRIAL)):
    os.mkdir(save.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, trial = TRIAL))


In [ ]:
def plot_history(hist, filename):
    plt.clf()
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.plot(hist[:,0], label='Train')
    plt.plot(hist[:,1], label = 'Validation')
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename)


In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(), loss = tf.keras.losses.mean_squared_error)


In [ ]:
# hist = []
for epoch in np.arange(10):
# for epoch in np.arange(MAX_EPOCH):
    for i in np.arange(int(TRAIN_SIZE / BATCH_SIZE)):
        dataset_valid = make_batch(VALID_INI - 1, VALID_END, int(1.0 * BATCH_SIZE * VALID_SIZE / TRAIN_SIZE))
        dataset_train = make_batch(TRAIN_INI - 1, TRAIN_END, BATCH_SIZE)
        history = model.fit(dataset_train, verbose = 0, epochs = 1, validation_data = dataset_valid, validation_steps = 1)
    hist = np.append(hist, [history.history['loss'], history.history['val_loss']])
hist = np.reshape(hist, (-1, 2))

        

In [ ]:

model.save(save.format(N = SYSTEM_SIZE, L = LENGTH_IN, K = COUPLING, label = LABEL, trial = TRIAL))

plot_history(hist, history_pdf.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, trial = TRIAL))

np.savetxt(history_dat.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, trial = TRIAL), hist, fmt = "%f")



In [ ]:

def test_same_init(offset, length_test):

    data = np.loadtxt(file.format(N = SYSTEM_SIZE, K = COUPLING, w = SHIFT, ft = FEET, dt = DT, e = 1))
    test_output = np.zeros((length_test, DIM), dtype=np.float32)

    test_image = data[offset:offset + LENGTH_IN]
    for i in np.arange(length_test):

        test_output[i] = model.predict(np.reshape(test_image, (1, DIM * LENGTH_IN)))[0]
        test_image = np.delete(test_image, 0, 0)
        test_image = np.append(test_image, np.reshape(test_output[i], (1, DIM)), axis=0)
        

    output_dat = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_same_{trial:d}.dat'
    np.savetxt(output_dat.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL), test_output, fmt = "%f")

    fig = plt.figure(figsize=(10,6))
    plt.subplot(3,1,1)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,2)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,3)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T - np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -2, vmax = 2, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout

    output_pdf = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_same_{trial:d}.pdf'
    plt.savefig(output_pdf.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL))




In [ ]:

def test_diff_init(offset, length_test):

    data = np.loadtxt(file.format(N = SYSTEM_SIZE, K = COUPLING, w = SHIFT, ft = FEET, dt = DT, e = 2))
    test_output = np.zeros((length_test, DIM), dtype=np.float32)

    test_image = data[offset:offset + LENGTH_IN]
    for i in np.arange(length_test):
        test_output[i] = model.predict(np.reshape(test_image, (1, DIM * LENGTH_IN)))[0]
        test_image = np.delete(test_image, 0, 0)
        test_image = np.append(test_image, np.reshape(test_output[i], (1, DIM)), axis=0)
        

    output_dat = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_diff_{trial:d}.dat'
    np.savetxt(output_dat.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL), test_output, fmt = "%f")

    fig = plt.figure(figsize=(10,6))
    plt.subplot(3,1,1)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,2)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,3)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T - np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -2, vmax = 2, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout

    output_pdf = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_diff_{trial:d}.pdf'
    plt.savefig(output_pdf.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL))




In [ ]:

def test_diff_coupling(offset, length_test):

    data = np.loadtxt('/pds/pds121/KMML/train_prediction/Phaset_w6.28_N4_K1.2000_ft1000_dt0.005_1.dat')
    test_output = np.zeros((length_test, DIM), dtype=np.float32)

    test_image = data[offset:offset + LENGTH_IN]
    for i in np.arange(length_test):
        test_output[i] = model.predict(np.reshape(test_image, (1, DIM * LENGTH_IN)))[0]
        test_image = np.delete(test_image, 0, 0)
        test_image = np.append(test_image, np.reshape(test_output[i], (1, DIM)), axis=0)
        

    output_dat = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_diff_{trial:d}.dat'
    np.savetxt(output_dat.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL), test_output, fmt = "%f")

    fig = plt.figure(figsize=(10,6))
    plt.subplot(3,1,1)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,2)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,3)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T - np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -2, vmax = 2, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout

    output_pdf = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_diff_{trial:d}.pdf'
    plt.savefig(output_pdf.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL))




In [ ]:

def test_diff_natural(offset, length_test):

    data = np.loadtxt('/pds/pds121/KMML/train_prediction/Phaset_w6.28_N4_K0.5000_ft1000_dt0.005_1.dat')
    test_output = np.zeros((length_test, DIM), dtype=np.float32)

    test_image = data[offset:offset + LENGTH_IN]
    for i in np.arange(length_test):
        test_output[i] = model.predict(np.reshape(test_image, (1, DIM * LENGTH_IN)))[0]
        test_image = np.delete(test_image, 0, 0)
        test_image = np.append(test_image, np.reshape(test_output[i], (1, DIM)), axis=0)
        

    output_dat = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_diff_{trial:d}.dat'
    np.savetxt(output_dat.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL), test_output, fmt = "%f")

    fig = plt.figure(figsize=(10,6))
    plt.subplot(3,1,1)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,2)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -1, vmax = 1, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout


    plt.subplot(3,1,3)
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.pcolor(data[offset:offset + LENGTH_IN + length_test].T - np.append(data[offset:offset + LENGTH_IN], test_output, axis = 0).T, vmin = -2, vmax = 2, cmap='Spectral')
    plt.colorbar()
    plt.tight_layout

    output_pdf = '/pds/pds151/ckj/MLKM/MLKM_Tracer_5/saves/N{N:d}K{K:0.4f}L{L:d}/Tracer_{label:d}_{offset:d}_{length:d}_output_diff_{trial:d}.pdf'
    plt.savefig(output_pdf.format(N = SYSTEM_SIZE, K = COUPLING, L = LENGTH_IN, label = LABEL, offset = offset, length = length_test, trial = TRIAL))


In [ ]:
# test_same_init(0, LENGTH_TEST)
# test_same_init(190000, LENGTH_TEST)
# test_diff_init(0, LENGTH_TEST)
# test_diff_init(100000, LENGTH_TEST)

In [ ]:
LENGTH_TEST = 400
test_same_init(0, LENGTH_TEST)

In [ ]:
test_same_init(190000, LENGTH_TEST)

In [ ]:
test_diff_init(0, LENGTH_TEST)

In [ ]:
test_diff_init(190000, LENGTH_TEST)

In [ ]:
test_diff_coupling(0, LENGTH_TEST)

In [ ]:
test_diff_coupling(190000, LENGTH_TEST)

In [ ]:
test_diff_natural(0, LENGTH_TEST)

In [ ]:
test_diff_natural(190000, LENGTH_TEST)